In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
#Read the key /home/julio/repos/nl2sparql_synthetic_data/data/openai_key.txt
with open('/home/julio/repos/nl2sparql_synthetic_data/data/openai_key.txt', 'r') as file:
    key = file.read().replace('\n', '')
os.environ["OPENAI_API_KEY"]= key

from openai import OpenAI

In [3]:
# os.chdir('../rdf_sensei')

In [4]:
from rdfsensei.semantic_search.data_loader import DataLoader
from rdfsensei.semantic_search.embedding_generator import EmbeddingGenerator
from rdfsensei.semantic_search.faiss_index import FaissIndex
from rdfsensei.semantic_search.query_processor import QueryProcessor
import numpy as np
import openai

/home/julio/envs/rdfsensei/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# data_directory = '../data/gendata_xcell'  # Directory where your JSON files are stored
data_directory = '/home/julio/repos/nl2sparql_synthetic_data/data/x_cell_small'  # Directory where your JSON files are stored

# Step 1: Load and preprocess data
data_loader = DataLoader(data_directory)
data_loader.load_data()
processed_data = data_loader.preprocess_questions()
if not processed_data:
    print("No data to process.")



Loading data from /home/julio/repos/nl2sparql_synthetic_data/data/x_cell_small/xcell_path_dataset_rating_k2.json


In [6]:

cleaned_questions, metadata = zip(*processed_data)

# Step 2: Generate embeddings
embedding_generator = EmbeddingGenerator()
embeddings = embedding_generator.generate_embeddings(list(cleaned_questions))


In [7]:

# Step 3: Build Faiss index
embedding_dim = embeddings.shape[1]
faiss_index = FaissIndex(embedding_dim, use_gpu=True)
faiss_index.build_index(embeddings, list(metadata))


In [10]:
labels_path = "/home/julio/repos/nl2sparql_synthetic_data/data/tabox_index/xsearch_cell/labels.obj"
counts_path = "/home/julio/repos/nl2sparql_synthetic_data/data/tabox_index/xsearch_cell/counts.obj"
endpoint_t_box_url = "https://knowledge.brc.riken.jp/bioresource/sparql"
database="http://metadb.riken.jp/db/xsearch_cell"
path_index = "/home/julio/repos/nl2sparql_synthetic_data/data/tabox_index/xsearch_cell/t_box_index/faiss"
# Step 4: Initialize Query Processor
from riqme.context.LLMStrategies import OpenAIStrategy  # Import your LLM strategy

# Load LLM strategy (can be OpenAI, Llama, etc.)
openai_client = OpenAI()
llm_strategy = OpenAIStrategy(openai_client)

query_processor = QueryProcessor(embedding_generator, 
                                 faiss_index, labels_path, 
                                 counts_path, endpoint_t_box_url, 
                                 database, path_index,llm_strategy=llm_strategy,
                                 model_name='gpt-4o'
                                 )



Loading labels on endpoint cache...
Labels on endpoint cache loaded!
Loading counts on endpoint cache...
Counts on endpoint cache loaded!
Loading existing FAISS index...
Existing index Loaded!


In [9]:

# Example usage
# user_query = "Which authority is cited by the cell line"
user_query = "Which cell line references the PubMed article 123d and what is its label?"
results = query_processor.process_query(user_query,top_k_questions=5,do_entity_linking=True)
# if result:
#     print("\nMatched Question:\n", result['original_question'])
#     print("\nSPARQL Query:\n", result['sparql_query'])
# else:
#     print("No similar question found.")
# for result in results['semantic_search']:
#     print("\nMatched Question:\n", result['original_question'])
#     print("\nSPARQL Query:\n", result['sparql_query'])
#     print('Distance:', result['distance'])
#     print('Entity mapping:', result['entity_mapping'])
print("--------SEMANTIC SEARCH--------")
print(results['semantic_search'])
print("--------ENTITY LINKING--------")
print(results['entity_linking'])

processing SPARQL query and answer: Which cell line references the PubMed article 123d and what is its label?
Added database to query: http://metadb.riken.jp/db/xsearch_cell
--------SEMANTIC SEARCH--------
[{'original_question': 'What is the <<label>> of the cell line <<AES0037_Freeze medium>> and its cited authority <<B6fc-1>> in the <<pubmed>> database?', 'sparql_query': 'SELECT ?label1 ?label2 ?authority WHERE {\n  <http://metadb.riken.jp/db/rikenbrc_cell/cell_49_b4331> <http://www.w3.org/2000/01/rdf-schema#label> ?label1 .\n  <http://metadb.riken.jp/db/rikenbrc_cell/cell_AES0037> <http://www.w3.org/2000/01/rdf-schema#label> ?label2 .\n  <http://metadb.riken.jp/db/rikenbrc_cell/cell_AES0037> <http://purl.org/net/cito/citesAsAuthority> ?authority .\n}', 'entity_mapping': {'AES0037_Freeze medium': '<http://metadb.riken.jp/db/rikenbrc_cell/cell_49_b4331>', 'B6fc-1': '<http://metadb.riken.jp/db/rikenbrc_cell/cell_AES0037>', 'label': '<http://www.w3.org/2000/01/rdf-schema#label>', 'pubme

In [13]:
# Example usage
# user_query = "Which authority is cited by the cell line"
user_query = " Which value of HPS0953_culture ?"
results = query_processor.process_query(user_query,top_k_questions=5,do_entity_linking=True)
# if result:
#     print("\nMatched Question:\n", result['original_question'])
#     print("\nSPARQL Query:\n", result['sparql_query'])
# else:
#     print("No similar question found.")
print("Best SPARQL Query:\n", results['best_sparql_query'])
print("--------SEMANTIC SEARCH--------")
for result in results['semantic_search']:
    print("\nMatched Question:\n", result['original_question'])
    print("\nSPARQL Query:\n", result['sparql_query'])
    print('Distance:', result['distance'])
    print('Entity mapping:', result['entity_mapping'])

print("--------ENTITY LINKING--------")
print(results['entity_linking'])
print("--------LLM ANSWER--------")
print(results['llm_answer']['answer'])


processing SPARQL query and answer:  Which value of HPS0953_culture ?
Added database to query: http://metadb.riken.jp/db/xsearch_cell
Best SPARQL Query:
 SELECT ?value WHERE {
  <http://metadb.riken.jp/db/rikenbrc_cell/cell_48_694> <http://metadb.riken.jp/db/rikenbrc_cell/cell_0000041> ?unknown .
  ?unknown <http://www.w3.org/1999/02/22-rdf-syntax-ns#value> ?value .
}
--------SEMANTIC SEARCH--------

Matched Question:
 Which value has been assigned to the <<HPS0953_culture condition>>?

SPARQL Query:
 SELECT ?value WHERE {
  <http://metadb.riken.jp/db/rikenbrc_cell/cell_48_694> <http://metadb.riken.jp/db/rikenbrc_cell/cell_0000041> ?unknown .
  ?unknown <http://www.w3.org/1999/02/22-rdf-syntax-ns#value> ?value .
}
Distance: 0.17061758
Entity mapping: {'HPS0953_culture condition': '<http://metadb.riken.jp/db/rikenbrc_cell/cell_48_694>'}

Matched Question:
 What numeric value is tied to the <<HPS0953_culture condition>>?

SPARQL Query:
 SELECT ?value WHERE {
  <http://metadb.riken.jp/db/